# CBOW(Continuous Bag-Of-Words) 코드 구현

# 
#### 네이버 영화리뷰를 사용하여 CBOW 구현
### 토큰화까지 참고한 블로그 https://wikidocs.net/44249

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

  
  네이버 영화리뷰  txt 파일 받아오기

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7fb88c77a460>)

In [3]:
train_data = pd.read_table('ratings.txt')

In [4]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [5]:
print(len(train_data)) # 리뷰 개수 출력

200000


train_data 의 데이터 중복 유무 확인 : 같은 문장인 것이 있는 경우 제거

In [6]:
# document 열과 label 열의 중복을 제외한 값의 개수
train_data['document'].nunique(), train_data['label'].nunique()

(194543, 2)

In [7]:
# document 열의 중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True)

In [8]:
print('총 샘플의 수 :',len(train_data))

총 샘플의 수 : 194544


In [9]:
print(train_data.groupby('label').size().reset_index(name = 'count'))

   label  count
0      0  97277
1      1  97267


In [10]:
print(train_data.isnull().values.any())

True


  
  다큐먼트에 아무것도 안쓴 리뷰가 1개 있어서 제거

In [11]:
print(train_data.isnull().sum())

id          0
document    1
label       0
dtype: int64


In [12]:
train_data.loc[train_data.document.isnull()]

,id,document,label
46471,6369843,NaN,1


In [13]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [14]:
print(len(train_data))

194543


In [15]:
#알파벳과 공백을 제외하고 모두 제거
eng_text = 'do!!! you expect... people~ to~ read~ the FAQ, etc. and actually accept hard~! atheism?@@'
print(re.sub(r'[^a-zA-Z ]', '', eng_text))

do you expect people to read the FAQ etc and actually accept hard atheism


  
    
  한글의 범위를 나타냄 : ㄱ-힣

In [16]:
# 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data[:5]

/tmp/ipykernel_427224/3525229848.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1


In [17]:
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

/tmp/ipykernel_427224/2260017223.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경


id             0
document    1025
label          0
dtype: int64


한글을 제외한 모든 것들 삭제

In [18]:
train_data.loc[train_data.document.isnull()][:5]

,id,document,label
203,10216462,NaN,1
608,8085167,NaN,1
661,3579401,NaN,1
984,5338464,NaN,1
1199,7887700,NaN,1


In [19]:
train_data = train_data.dropna(how = 'any')
print(len(train_data))

193518


In [20]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [21]:
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

['오다', '이렇다', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만들다', '게', '나다', '뻔']

In [22]:
X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

100%|██████████| 193518/193518 [11:06<00:00, 290.56it/s]


In [23]:
print(X_train[:3])

[['어리다', '때', '보고', '지금', '다시', '보다', '재밌다', 'ㅋㅋ'], ['디자인', '을', '배우다', '학생', '외국', '디자이너', '그', '일군', '전통', '을', '통해', '발전', '문화', '산업', '부럽다', '사실', '우리나라', '에서도', '그', '어렵다', '시절', '끝', '까지', '열정', '을', '지키다', '노라노', '같다', '전통', '있다', '저', '같다', '사람', '꿈', '을', '꾸다', '이루다', '나가다', '수', '있다', '것', '감사하다'], ['폴리스스토리', '시리즈', '부터', '뉴', '까지', '버리다', '하나', '없다', '최고']]


토큰화 완료됨

In [24]:
sentences = X_train
print(np.shape(sentences))
print(sentences[3])

(193518,)
['오다', '연기', '진짜', '개', '쩔다', '지루하다', '생각', '몰입', '보다', '그렇다', '이렇다', '진짜', '영화', '지']


/usr/local/lib/python3.8/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


토큰나이저

In [25]:
import tensorflow as tf

In [26]:
import numpy as np

# 단어 사전 생성
word2idx = {}  # 단어 사전
idx2word = {}  # 인덱스 -> 단어 매핑
for sentence in sentences:
    for word in sentence:
        if word not in word2idx:
            idx = len(word2idx)
            word2idx[word] = idx
            idx2word[idx] = word

vocab_size = len(word2idx)
print(vocab_size)

49645


In [27]:
print(idx2word)

{0: '어리다', 1: '때', 2: '보고', 3: '지금', 4: '다시', 5: '보다', 6: '재밌다', 7: 'ㅋㅋ', 8: '디자인', 9: '을', 10: '배우다', 11: '학생', 12: '외국', 13: '디자이너', 14: '그', 15: '일군', 16: '전통', 17: '통해', 18: '발전', 19: '문화', 20: '산업', 21: '부럽다', 22: '사실', 23: '우리나라', 24: '에서도', 25: '어렵다', 26: '시절', 27: '끝', 28: '까지', 29: '열정', 30: '지키다', 31: '노라노', 32: '같다', 33: '있다', 34: '저', 35: '사람', 36: '꿈', 37: '꾸다', 38: '이루다', 39: '나가다', 40: '수', 41: '것', 42: '감사하다', 43: '폴리스스토리', 44: '시리즈', 45: '부터', 46: '뉴', 47: '버리다', 48: '하나', 49: '없다', 50: '최고', 51: '오다', 52: '연기', 53: '진짜', 54: '개', 55: '쩔다', 56: '지루하다', 57: '생각', 58: '몰입', 59: '그렇다', 60: '이렇다', 61: '영화', 62: '지', 63: '안개', 64: '자욱하다', 65: '밤하늘', 66: '뜨다', 67: '초승달', 68: '사랑', 69: '해보다', 70: '라면', 71: '처음', 72: '웃다', 73: '완전', 74: '감동', 75: '이다', 76: '전쟁', 77: '나오다', 78: '빠', 79: '로', 80: '싶다', 81: '굿', 82: '바보', 83: '아니다', 84: '병', 85: '쉰', 86: '내', 87: '나이', 88: '나', 89: '적', 90: '하지만', 91: '훗날', 92: '사하나', 93: '하', 94: '감정', 95: '완벽하다', 96: '이해', 97: '고질', 98: '라니', 9

In [28]:
print(word2idx)

{'어리다': 0, '때': 1, '보고': 2, '지금': 3, '다시': 4, '보다': 5, '재밌다': 6, 'ㅋㅋ': 7, '디자인': 8, '을': 9, '배우다': 10, '학생': 11, '외국': 12, '디자이너': 13, '그': 14, '일군': 15, '전통': 16, '통해': 17, '발전': 18, '문화': 19, '산업': 20, '부럽다': 21, '사실': 22, '우리나라': 23, '에서도': 24, '어렵다': 25, '시절': 26, '끝': 27, '까지': 28, '열정': 29, '지키다': 30, '노라노': 31, '같다': 32, '있다': 33, '저': 34, '사람': 35, '꿈': 36, '꾸다': 37, '이루다': 38, '나가다': 39, '수': 40, '것': 41, '감사하다': 42, '폴리스스토리': 43, '시리즈': 44, '부터': 45, '뉴': 46, '버리다': 47, '하나': 48, '없다': 49, '최고': 50, '오다': 51, '연기': 52, '진짜': 53, '개': 54, '쩔다': 55, '지루하다': 56, '생각': 57, '몰입': 58, '그렇다': 59, '이렇다': 60, '영화': 61, '지': 62, '안개': 63, '자욱하다': 64, '밤하늘': 65, '뜨다': 66, '초승달': 67, '사랑': 68, '해보다': 69, '라면': 70, '처음': 71, '웃다': 72, '완전': 73, '감동': 74, '이다': 75, '전쟁': 76, '나오다': 77, '빠': 78, '로': 79, '싶다': 80, '굿': 81, '바보': 82, '아니다': 83, '병': 84, '쉰': 85, '내': 86, '나이': 87, '나': 88, '적': 89, '하지만': 90, '훗날': 91, '사하나': 92, '하': 93, '감정': 94, '완벽하다': 95, '이해': 96, '고질': 97, '라니': 98, '

In [29]:
# 학습 데이터 전처리
X_train = []
Y_train = []
window_size = 2

for sentence in sentences:
    for i, word in enumerate(sentence):
        context_words = []
        target_word = []
        for j in range(i - window_size, i + window_size + 1):
            if j != i and j >= 0 and j < len(sentence):
                context_words.append(sentence[j])
        target_word.append(word)
        X_train.append(context_words)
        Y_train.append(target_word)

# 입력 데이터와 출력 데이터를 정수 인코딩으로 변환
X_train = [[word2idx[word] for word in context] for context in X_train]
Y_train = [[word2idx[word] for word in target] for target in Y_train]

In [30]:
idx_list = [idx for idx, row in enumerate(X_train[:500]) if len(row) != 4]

print(idx_list)  # [1, 2, 3, 4]

[0, 1, 6, 7, 8, 9, 48, 49, 50, 51, 57, 58, 59, 60, 71, 72, 73, 74, 79, 80, 81, 82, 91, 92, 93, 94, 97, 98, 99, 100, 105, 106, 107, 108, 109, 111, 112, 113, 114, 134, 135, 136, 137, 138, 141, 142, 143, 144, 154, 155, 156, 157, 158, 159, 160, 174, 175, 176, 177, 185, 186, 187, 188, 189, 190, 196, 197, 198, 199, 214, 215, 216, 217, 221, 222, 223, 224, 256, 257, 258, 259, 261, 262, 263, 264, 269, 270, 271, 272, 275, 276, 277, 278, 305, 306, 307, 308, 317, 318, 319, 320, 336, 337, 338, 339, 346, 347, 348, 349, 361, 362, 363, 364, 369, 370, 371, 372, 373, 374, 375, 376, 385, 386, 387, 388, 391, 392, 393, 394, 397, 398, 399, 400, 402, 403, 404, 405, 409, 410, 411, 412, 414, 415, 416, 417, 423, 424, 425, 426, 435, 436, 437, 438, 441, 442, 443, 444, 448, 449, 450, 451, 460, 461, 462, 463, 465, 466, 467, 468, 472, 473, 474, 475]


In [31]:
num = 423
for q,j in enumerate(X_train[num:num+5]):
    print("Context words :" ,j)
    for i in j:
        print(idx2word[i])
    print("Center words :", Y_train[num+q])
    print(idx2word[num+q])
    print("\n")

Context words : [294, 77, 279]
많이
나오다
재미있다
Center words : [7]
이영자


Context words : [77, 7]
나오다
ㅋㅋ
Center words : [279]
함


Context words : [296, 232]
바다
속
Center words : [295]
대박


Context words : [295, 232, 297]
마치
속
아쿠아리움
Center words : [296]
일텐데


Context words : [295, 296, 297, 232]
마치
바다
아쿠아리움
속
Center words : [232]
왠




In [34]:
import tensorflow.compat.v1 as tf1
# import tensorflow as tf2
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.05
session = tf1.Session(config=config)
from tensorflow.keras import layers


2023-03-17 09:26:31.887769: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4055 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:b3:00.0, compute capability: 8.0


In [35]:
class multiplyOperat(layers.Layer):
    def __init__(self, units=None, init_value=1, **kwargs):
        super(multiplyOperat, self).__init__()
        self.units = units
        self.init_value = tf.cast(init_value, dtype=tf.float32)
    def build(self, input_shape):
        if not self.units is None:
            a_init = tf.Variable(tf.fill(self.units, self.init_value), dtype=tf.float32)
        else:
            a_init = tf.Variable(tf.fill(input_shape[1:], self.init_value), dtype=tf.float32)
        self.a = tf.Variable(initial_value=a_init, trainable=True, name="a_p")
        super(multiplyOperat, self).build(input_shape)

    def call(self, inputs, **kwargs):
        return tf.multiply(inputs, self.a)

    def get_config(self):
        config = {'units': self.units}
        base_config = super(multiplyOperat, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    #@tf_utils.shape_type_conversion
    def compute_output_shape(self, input_shape):
        return input_shape
class shiftOperat(layers.Layer):
    def __init__(self, units=None, init_value=0, **kwargs):
        super(shiftOperat, self).__init__()
        self.units = units
        self.init_value = tf.cast(init_value, dtype=tf.float32)

    def build(self, input_shape):
        if not self.units is None:
            s_init = tf.Variable(tf.fill(self.units, self.init_value), dtype=tf.float32)
        else:
            s_init = tf.Variable(tf.fill(input_shape[1:], self.init_value), dtype=tf.float32)
        self.s = tf.Variable(initial_value=s_init, trainable=True, name="s_p")
        super(shiftOperat, self).build(input_shape)

    def call(self, inputs, **kwargs):
        return tf.subtract(inputs, self.s)

    def get_config(self):
        config = {'units': self.units}
        base_config = super(shiftOperat, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    #@tf_utils.shape_type_conversion
    def compute_output_shape(self, input_shape):
        return input_shape
class abTanh(layers.Layer):
    def __init__(self, units=None, init_a=1, init_b=0, is_trainable=True, **kwargs):
        super(abTanh, self).__init__()
        self.units = units
        self.init_a = init_a
        self.init_b = init_b
        self.trainable = is_trainable

    def build(self, input_shape):
        self.shift = shiftOperat(units=self.units, init_value=self.init_b)
        self.activation = tf.nn.tanh
        self.multiply = multiplyOperat(units=self.units, init_value=self.init_a)
        super(abTanh, self).build(input_shape)

    def call(self, inputs, **kwargs):
        out = tf.cast(inputs, dtype=tf.float32)
        out = self.shift(inputs=out)
        out = self.activation(out)
        out = self.multiply(inputs=out)
        return out

    def get_config(self):
        config = {'units': self.units}
        base_config = super(abTanh, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    #@tf_utils.shape_type_conversion
    def compute_output_shape(self, input_shape):
        return input_shape

In [36]:
import tensorflow as tf
from tensorflow.keras import layers

class CBOWModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, window_size):
        super(CBOWModel, self).__init__()
        self.embeddings = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        self.pooling = layers.GlobalAveragePooling1D()
        self.dense1 = layers.Dense(units=embedding_dim)
        self.a1 = abTanh()
        
        self.dense2 = layers.Dense(units=vocab_size, activation='softmax')
        self.window_size = window_size

    def call(self, input_word_indices):
        # 입력으로 받은 인덱스 리스트에 대해 임베딩 수행
        embedded_words = self.embeddings(input_word_indices)
        
        # 입력 윈도우 크기에 따라 해당 범위 내의 단어 임베딩에 대해 평균값 계산
        pooled_words = self.pooling(embedded_words[:, :self.window_size, :])
        out = self.dense1(pooled_words)
        out = self.a1(out)
        
        # 예측 벡터 생성
        output = self.dense2(out)
        
        return output

In [37]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post')
y_train = tf.keras.preprocessing.sequence.pad_sequences(Y_train, padding='post')

x_train = tf.convert_to_tensor(x_train)
y_train = tf.convert_to_tensor(y_train)

2023-03-17 09:29:20.094907: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4055 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:b3:00.0, compute capability: 8.0


In [44]:
print(np.shape(x_train))

(2133859, 4)


In [38]:
# 단어 인덱스 리스트를 입력으로 받는 CBOW 모델 생성
model = CBOWModel(vocab_size=vocab_size, embedding_dim=128, window_size=2)

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
    6/66684 [..............................] - ETA: 26:57 - loss: 10.8117 - accuracy: 0.0104      

2023-03-17 09:29:52.329355: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


66684/66684 [==============================] - 1645s 25ms/step - loss: 7.0456 - accuracy: 0.0938
Epoch 2/10
66684/66684 [==============================] - 1641s 25ms/step - loss: 6.6579 - accuracy: 0.1184
Epoch 3/10
66684/66684 [==============================] - 1058s 16ms/step - loss: 6.4791 - accuracy: 0.1277
Epoch 4/10
66684/66684 [==============================] - 980s 15ms/step - loss: 6.3611 - accuracy: 0.1335
Epoch 5/10
66684/66684 [==============================] - 969s 15ms/step - loss: 6.2756 - accuracy: 0.1380
Epoch 6/10
66684/66684 [==============================] - 978s 15ms/step - loss: 6.2104 - accuracy: 0.1415
Epoch 7/10
66684/66684 [==============================] - 981s 15ms/step - loss: 6.1700 - accuracy: 0.1441
Epoch 8/10
66684/66684 [==============================] - 982s 15ms/step - loss: 6.1353 - accuracy: 0.1467
Epoch 9/10
66684/66684 [==============================] - 980s 15ms/step - loss: 6.0966 - accuracy: 0.1497
Epoch 10/10
66684/66684 [====================

In [40]:
import os
# 저장할 폴더 경로
save_dir = '/home/oem/KYM/트랜스포머를 활용한 자연어 처리/CBOW/Parametric/'
os.makedirs(save_dir, exist_ok=True)
# 모델 저장
model.save_weights(os.path.join(save_dir, 'my_word2vec_model.h5'))

In [41]:
# 임베딩 행렬 출력
embedding_matrix = model.layers[0].get_weights()[0]
print(embedding_matrix)

[[ 0.30483007 -0.2185761   0.3100326  ... -0.01757209  0.2898982
  -0.08133964]
 [ 0.36899    -0.10360066 -0.32377252 ...  0.06276515 -0.26183993
  -0.04297026]
 [ 0.30442527  0.4118986  -0.09405338 ...  0.12416028  0.19072168
  -0.06004051]
 ...
 [-0.36394867 -0.08172075  0.37551084 ...  0.26850036  0.01529878
   0.286335  ]
 [ 0.23472573 -0.46677217  0.1682862  ... -0.32021132  0.4217928
   0.5819303 ]
 [-0.2096321  -0.4397367  -0.20415692 ... -0.04552736 -0.10067488
   0.10093978]]


In [45]:
from tensorflow.keras.models import load_model
import os
import numpy as np

# 하이퍼파라미터
embedding_dim = 128  # 임베딩 차원 수
window_size = 2  # 주변 단어 개수

# 저장된 모델 불러오기
save_dir = '/home/oem/KYM/트랜스포머를 활용한 자연어 처리/CBOW/Parametric/'
model_path = os.path.join(save_dir, 'my_word2vec_model.h5')

model = CBOWModel(vocab_size=vocab_size, embedding_dim=128, window_size=2)

inputs = tf.keras.Input(shape=(4,))
_ = model(inputs)
model.load_weights(model_path)

128차원 첫 번째 임베딩 층(선형변환)

In [56]:
print(model.layers[0].get_weights()[0])
print(np.shape(model.layers[0].get_weights()[0]))

[[ 0.30483007 -0.2185761   0.3100326  ... -0.01757209  0.2898982
  -0.08133964]
 [ 0.36899    -0.10360066 -0.32377252 ...  0.06276515 -0.26183993
  -0.04297026]
 [ 0.30442527  0.4118986  -0.09405338 ...  0.12416028  0.19072168
  -0.06004051]
 ...
 [-0.36394867 -0.08172075  0.37551084 ...  0.26850036  0.01529878
   0.286335  ]
 [ 0.23472573 -0.46677217  0.1682862  ... -0.32021132  0.4217928
   0.5819303 ]
 [-0.2096321  -0.4397367  -0.20415692 ... -0.04552736 -0.10067488
   0.10093978]]
(49645, 128)


abTanh 의 비선형변환된 128차원 임베딩 층
문제는 , 토큰인코딩(원-핫) 에서 워드 임베딩으로 간 경우는 인덱싱이 가능하나, 그 후부터는 해석이 섞임 

In [59]:
print(model.layers[2].get_weights()[0])
print(np.shape(model.layers[2].get_weights()[0]))

[[ 0.18428057  0.28093234 -0.06117422 ...  0.7597251  -0.21017838
  -0.0384725 ]
 [ 0.15316348  1.1496726   0.81946224 ...  0.36414775 -0.0571945
  -0.45005098]
 [ 0.10683222 -0.26789936  0.72148603 ...  0.42896888 -0.06233373
  -0.08553268]
 ...
 [-0.05540627  0.7079064   0.0020869  ... -0.5483162   0.05198722
   0.655279  ]
 [-0.20552336 -0.68926364  0.19577065 ... -0.09157666  0.09724931
  -0.15140636]
 [-0.00457441 -0.48597386 -0.21307044 ...  0.5654837  -0.10698918
   0.25323716]]
(128, 128)


다시 선형변환된 128차원 임베딩 2번째 층

In [55]:
print(model.layers[3].get_weights()[0])

[-0.45220375 -0.24993841  0.11778243  0.12643802 -0.3847001  -0.37579858
 -0.2872316   0.11539312 -0.11390284 -0.45182478  0.3596382  -0.3639024
  0.3179464  -0.86958337 -0.46384484 -0.0907532  -0.4251848  -0.02749178
 -0.2541569   0.7547267  -0.22729386  0.3508441   0.31265464 -0.51877475
 -0.3805474   0.35865584 -0.24502617 -0.12726937 -0.39496475  0.00095261
  0.22702351 -0.19305465  0.34509724  0.38808876  0.24339955  0.11050659
 -0.20881942 -0.18837643 -0.13273789 -0.39495322  0.16056706 -0.16367376
 -0.15532058  0.26612023  0.16571143 -0.34777007 -0.34061044 -0.19081104
  0.2454302   0.24967857  0.06776443 -0.4292047  -0.30556533 -0.34833816
  0.38449422  0.2737127  -0.08982935 -0.06156638  0.12983747 -0.24395014
 -0.19718055  0.5721814   0.1330526   0.31744534  0.16944562  0.26196998
 -0.30432782  0.26967642  0.06078332 -0.33911714 -0.17656152 -0.37767625
  0.21020369  0.38434148 -0.13159603 -0.27511203 -0.46442896  0.18567868
 -0.14399734  0.28678277 -0.303352    0.31799188 -0.

In [60]:
def find_similar_rows(matrix, row_index, num_similar_rows=6):
    # 특정 행 가져오기
    row = matrix[row_index]

    # 모든 행과의 유사도 계산하기
    similarities = np.dot(matrix, row) / (np.linalg.norm(matrix, axis=1) * np.linalg.norm(row))

    # 유사도가 높은 상위 N개의 인덱스 찾기
    similar_indices = np.argsort(similarities)[::-1][:num_similar_rows]
    similar_values = similarities[similar_indices]

    return similar_indices[1:], similar_values[1:]

In [63]:
word2idx_list = word2idx

In [64]:
new_input = input("단어를 입력하면 가까운 단어를 구해줌 : \n")
new_input = okt.morphs(new_input, stem = True)
print("\n불용어 처리로 바뀐 단어 : ",new_input,'\n')


for i in np.arange(len(new_input)):
    print("==============================================================")
    if new_input[i] in idx2word.values():
        print("{} 라는 단어가 토큰 사전에 있음".format(new_input[i]))
        
        encoding_one_hot = word2idx_list[new_input[i]]
        print(new_input[i], " 를 나타내는 토큰번호 : " ,encoding_one_hot,'\n')
        
        embed_input = embedding_matrix[encoding_one_hot]
        sim_words,sim_val = find_similar_rows(embedding_matrix, row_index=encoding_one_hot)
        print("{} 와 유사하다고 판단된 단어 : ".format(new_input[i]))
        for j,k in enumerate(sim_words):
            print(j+1,"순위 : ", idx2word[k], "// 유사도 : ", sim_val[j])
        print('\n\n')
        

    else:
        print("{} 라는 단어가 토큰 사전에 없음".format(new_input[i]))
        print("\n\n")

단어를 입력하면 가까운 단어를 구해줌 : 
질질 짜면서 보다가 누가 갑자기 들어와서 깜짝 놀랐음

불용어 처리로 바뀐 단어 :  ['질질', '짜다', '보다', '누가', '갑자기', '들어오다', '깜짝', '놀라다'] 

질질 라는 단어가 토큰 사전에 있음
질질  를 나타내는 토큰번호 :  4129 

질질 와 유사하다고 판단된 단어 : 
1 순위 :  하달 // 유사도 :  0.5306231
2 순위 :  패불 // 유사도 :  0.51812965
3 순위 :  맹렬하다 // 유사도 :  0.50528586
4 순위 :  꼬듯 // 유사도 :  0.4713198
5 순위 :  뭐왜케 // 유사도 :  0.46652842



짜다 라는 단어가 토큰 사전에 있음
짜다  를 나타내는 토큰번호 :  200 

짜다 와 유사하다고 판단된 단어 : 
1 순위 :  불량하다 // 유사도 :  0.45528385
2 순위 :  패스트푸드 // 유사도 :  0.42799386
3 순위 :  운세 // 유사도 :  0.42038187
4 순위 :  차제 // 유사도 :  0.4074329
5 순위 :  끼우다 // 유사도 :  0.40430936



보다 라는 단어가 토큰 사전에 있음
보다  를 나타내는 토큰번호 :  5 

보다 와 유사하다고 판단된 단어 : 
1 순위 :  비숍 // 유사도 :  0.54055524
2 순위 :  만들다 // 유사도 :  0.4658566
3 순위 :  보다도 // 유사도 :  0.46321124
4 순위 :  파보 // 유사도 :  0.45185676
5 순위 :  끌껄 // 유사도 :  0.44780973



누가 라는 단어가 토큰 사전에 있음
누가  를 나타내는 토큰번호 :  1346 

누가 와 유사하다고 판단된 단어 : 
1 순위 :  그른 // 유사도 :  0.44204247
2 순위 :  훨남 // 유사도 :  0.42796472
3 순위 :  티져 // 유사도 :  0.42646787
4 순위 :  데꼬 // 유사도